In [ ]:
!pip install gradio PyPDF2 reportlab

In [ ]:
import gradio as gr
import PyPDF2
from PyPDF2 import PdfWriter, PdfReader
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.colors import Color
from reportlab.lib.units import inch
import io
import os
import tempfile
from typing import List, Tuple, Optional
import zipfile

# 📚 Libraries Used in This Notebook

This notebook uses several important libraries to build a **Personal iLovePDF Clone** in Google Colab.  
Here’s what each one is used for in general, and how it is used in **my code**.

---

## 🔹 1. `gradio`
A Python library for building interactive **web-based UIs** for Python code.  
It makes it easy to create interfaces with buttons, file uploads, sliders, and textboxes.  

**In my code:**
- used to create the **Colab-friendly interface** for PDF tools.  
- provides:
  - file upload boxes for selecting PDFs  
  - buttons to trigger operations (merge, split, compress, etc.)  
  - textboxes and sliders for user input (e.g., watermark text, opacity, page numbers).  
- turns all backend functions into a **user-friendly PDF toolkit app**.

---

## 🔹 2. `PyPDF2`
A popular Python library for reading, writing, and modifying PDF files.  

**In my code:**
- used for all PDF operations:
  - `PdfReader` → read pages from a PDF.  
  - `PdfWriter` → create new PDFs after modifications.  
- enables:
  - merging PDFs  
  - splitting PDFs  
  - removing or rearranging pages  
  - rotating pages  
  - extracting text.  

This is the **core library that manipulates PDF content**.

---

## 🔹 3. `reportlab`
A library for programmatically creating and editing PDF files.  

**In my code:**
- used specifically for **adding watermarks**:
  - generates a transparent page with watermark text (using fonts, colors, opacity).  
  - then merges this watermark page onto existing PDFs.  
- handles text styling, positioning, and rotation (diagonal watermark).

---

## 🔹 4. `io`
Python’s built-in module for working with **streams of data in memory**.  

**In my code:**
- used with `io.BytesIO()` to create a temporary in-memory file for the watermark.  
- avoids saving intermediate files to disk, improving speed and memory usage.  

---

## 🔹 5. `os`
Python’s standard library for interacting with the operating system.  

**In my code:**
- used for:
  - deleting temporary files after use (`os.unlink`).  
  - checking file sizes (useful for compression size reduction report).  

---

## 🔹 6. `tempfile`
Python’s built-in module for creating **temporary files and directories**.  

**In my code:**
- used to generate temporary PDF or ZIP files while processing.  
- ensures unique, safe file paths inside Colab’s runtime.  
- prevents filename conflicts when handling multiple PDFs.  

---

## 🔹 7. `typing` (`List`, `Tuple`, `Optional`)
Python’s type hinting system for writing **readable and maintainable code**.  

**In my code:**
- used for type annotations in function signatures:
  - `List` → list of uploaded files.  
  - `Tuple[Optional[str], str]` → return value (path to processed file, status message).  
  - `Optional` → means file path might be `None` if an error occurs.  
- improves code clarity and debugging.  

---

## 🔹 8. `zipfile`
Python’s built-in module for creating and extracting ZIP archives.  

**In my code:**
- used in the **Split PDF** feature:  
  - each split PDF is generated and then stored inside a single `.zip` file.  
  - this makes downloading multiple split parts much easier.  

---

✨ With these libraries working together:  
- `PyPDF2` + `reportlab` handle **PDF manipulation**,  
- `gradio` creates the **user interface**,  
- `io`, `os`, `tempfile`, and `zipfile` handle **files and storage**,  
- `typing` keeps the code **clean and structured**.  


In [ ]:
class PDFToolkit:
    def __init__(self):
        pass

    def merge_pdfs(self, files: List) -> Tuple[Optional[str], str]:
        """Merge multiple PDF files into one"""
        if not files or len(files) < 2:
            return None, "Please upload at least 2 PDF files to merge."

        try:
            merger = PdfWriter()

            for file in files:
                reader = PdfReader(file.name)
                for page in reader.pages:
                    merger.add_page(page)

            output_path = tempfile.mktemp(suffix='.pdf')
            with open(output_path, 'wb') as output_file:
                merger.write(output_file)

            return output_path, "PDFs merged successfully!"

        except Exception as e:
            return None, f"Error merging PDFs: {str(e)}"

    def split_pdf(self, file, pages_per_split: int = 1) -> Tuple[Optional[str], str]:
        """Split PDF into individual pages or groups of pages"""
        if not file:
            return None, "Please upload a PDF file to split."

        try:
            reader = PdfReader(file.name)
            total_pages = len(reader.pages)

            if pages_per_split <= 0:
                pages_per_split = 1

            # Create a zip file containing all split PDFs
            zip_path = tempfile.mktemp(suffix='.zip')

            with zipfile.ZipFile(zip_path, 'w') as zipf:
                for i in range(0, total_pages, pages_per_split):
                    writer = PdfWriter()

                    # Add pages to current split
                    end_page = min(i + pages_per_split, total_pages)
                    for j in range(i, end_page):
                        writer.add_page(reader.pages[j])

                    # Write to temporary file
                    split_path = tempfile.mktemp(suffix='.pdf')
                    with open(split_path, 'wb') as split_file:
                        writer.write(split_file)

                    # Add to zip
                    split_name = f"split_{i+1}-{end_page}.pdf"
                    zipf.write(split_path, split_name)
                    os.unlink(split_path)

            return zip_path, f"PDF split into {(total_pages + pages_per_split - 1) // pages_per_split} files!"

        except Exception as e:
            return None, f"Error splitting PDF: {str(e)}"

    def remove_pages(self, file, pages_to_remove: str) -> Tuple[Optional[str], str]:
        """Remove specific pages from PDF"""
        if not file:
            return None, "Please upload a PDF file."

        try:
            reader = PdfReader(file.name)
            total_pages = len(reader.pages)

            # Parse pages to remove
            pages_to_remove_list = []
            for page_range in pages_to_remove.split(','):
                page_range = page_range.strip()
                if '-' in page_range:
                    start, end = map(int, page_range.split('-'))
                    pages_to_remove_list.extend(range(start-1, end))  # Convert to 0-based
                else:
                    pages_to_remove_list.append(int(page_range) - 1)  # Convert to 0-based

            writer = PdfWriter()
            for i, page in enumerate(reader.pages):
                if i not in pages_to_remove_list:
                    writer.add_page(page)

            output_path = tempfile.mktemp(suffix='.pdf')
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)

            return output_path, f"Removed pages: {pages_to_remove}. New PDF has {len(writer.pages)} pages."

        except Exception as e:
            return None, f"Error removing pages: {str(e)}"

    def compress_pdf(self, file) -> Tuple[Optional[str], str]:
        """Basic PDF compression by rewriting content"""
        if not file:
            return None, "Please upload a PDF file to compress."

        try:
            reader = PdfReader(file.name)
            writer = PdfWriter()

            for page in reader.pages:
                # Compress content streams
                page.compress_content_streams()
                writer.add_page(page)

            output_path = tempfile.mktemp(suffix='.pdf')
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)

            # Calculate size reduction
            original_size = os.path.getsize(file.name)
            compressed_size = os.path.getsize(output_path)
            reduction = (1 - compressed_size / original_size) * 100

            return output_path, f"PDF compressed! Size reduced by {reduction:.1f}%"

        except Exception as e:
            return None, f"Error compressing PDF: {str(e)}"

    def rotate_pages(self, file, rotation_angle: int, page_range: str = "all") -> Tuple[Optional[str], str]:
        """Rotate pages in PDF"""
        if not file:
            return None, "Please upload a PDF file to rotate."

        try:
            reader = PdfReader(file.name)
            writer = PdfWriter()
            total_pages = len(reader.pages)

            # Parse page range
            if page_range.lower() == "all":
                pages_to_rotate = list(range(total_pages))
            else:
                pages_to_rotate = []
                for page_spec in page_range.split(','):
                    page_spec = page_spec.strip()
                    if '-' in page_spec:
                        start, end = map(int, page_spec.split('-'))
                        pages_to_rotate.extend(range(start-1, end))
                    else:
                        pages_to_rotate.append(int(page_spec) - 1)

            for i, page in enumerate(reader.pages):
                if i in pages_to_rotate:
                    page.rotate(rotation_angle)
                writer.add_page(page)

            output_path = tempfile.mktemp(suffix='.pdf')
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)

            return output_path, f"Pages rotated by {rotation_angle} degrees!"

        except Exception as e:
            return None, f"Error rotating pages: {str(e)}"

    def extract_text(self, file) -> str:
        """Extract text from PDF"""
        if not file:
            return "Please upload a PDF file to extract text from."

        try:
            reader = PdfReader(file.name)
            text = ""

            for page_num, page in enumerate(reader.pages):
                text += f"\n--- Page {page_num + 1} ---\n"
                text += page.extract_text()
                text += "\n"

            return text.strip()

        except Exception as e:
            return f"Error extracting text: {str(e)}"

    def add_watermark(self, file, watermark_text: str, opacity: float = 0.3) -> Tuple[Optional[str], str]:
        """Add text watermark to PDF"""
        if not file:
            return None, "Please upload a PDF file to add watermark."

        if not watermark_text.strip():
            return None, "Please enter watermark text."

        try:
            reader = PdfReader(file.name)
            writer = PdfWriter()

            # Create watermark
            watermark_buffer = io.BytesIO()
            c = canvas.Canvas(watermark_buffer, pagesize=letter)

            # Set watermark properties - using RGB with opacity
            c.setFillColorRGB(0.5, 0.5, 0.5, alpha=opacity)  # Gray color with opacity
            c.setFont("Helvetica-Bold", 50)

            # Get text width for centering
            text_width = c.stringWidth(watermark_text, "Helvetica-Bold", 50)

            # Add watermark text diagonally across the page
            c.saveState()
            c.translate(letter[0]/2, letter[1]/2)  # Center of page
            c.rotate(45)  # 45 degree rotation
            c.drawString(-text_width/2, 0, watermark_text)  # Center the text
            c.restoreState()
            c.save()

            watermark_buffer.seek(0)
            watermark_reader = PdfReader(watermark_buffer)
            watermark_page = watermark_reader.pages[0]

            # Apply watermark to each page
            for page in reader.pages:
                page.merge_page(watermark_page)
                writer.add_page(page)

            output_path = tempfile.mktemp(suffix='.pdf')
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)

            return output_path, f"Watermark '{watermark_text}' added successfully!"

        except Exception as e:
            return None, f"Error adding watermark: {str(e)}"

    def rearrange_pages(self, file, new_order: str) -> Tuple[Optional[str], str]:
        """Rearrange pages in PDF according to specified order"""
        if not file:
            return None, "Please upload a PDF file to rearrange."

        try:
            reader = PdfReader(file.name)
            total_pages = len(reader.pages)

            # Parse new order
            page_order = []
            for page_num in new_order.split(','):
                page_num = page_num.strip()
                if page_num.isdigit():
                    page_idx = int(page_num) - 1  # Convert to 0-based
                    if 0 <= page_idx < total_pages:
                        page_order.append(page_idx)
                    else:
                        return None, f"Page {page_num} is out of range. PDF has {total_pages} pages."

            if not page_order:
                return None, "Please specify a valid page order."

            writer = PdfWriter()
            for page_idx in page_order:
                writer.add_page(reader.pages[page_idx])

            output_path = tempfile.mktemp(suffix='.pdf')
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)

            return output_path, f"Pages rearranged! New order: {new_order}"

        except Exception as e:
            return None, f"Error rearranging pages: {str(e)}"

# 🛠️ `PDFToolkit` Class

The **`PDFToolkit`** class is a custom-built utility that provides a wide range of PDF manipulation features.  
It acts like a personal version of **iLovePDF**, running inside Google Colab or Jupyter with an easy-to-use interface.

---

## 🔹 Class Overview
- Encapsulates all PDF-related functions into one toolkit.  
- Each method performs a specific PDF operation (merge, split, remove pages, etc.).  
- Returns:
  - `output_path`: a temporary file path for the processed PDF.  
  - `status message`: a string describing the operation result.  

---

## 🔹 1. `__init__`
**Purpose:**  
Constructor method for initializing the toolkit.  

**In my code:**
- Currently empty (`pass`), but provides a base to extend later (e.g., add default settings).

---

## 🔹 2. `merge_pdfs(files)`
**Purpose:**  
Combine **multiple PDF files** into a single PDF.  

**Steps in my code:**
- Checks that at least 2 files are uploaded.  
- Reads each file with `PdfReader`.  
- Iterates through all pages and adds them into a `PdfWriter`.  
- Writes the merged PDF to a temporary file.  

✅ Output: **one combined PDF**.

---

## 🔹 3. `split_pdf(file, pages_per_split=1)`
**Purpose:**  
Split a PDF into smaller files (per page or in groups).  

**Steps in my code:**
- Reads total pages of the PDF.  
- Creates a `.zip` archive to store all the split files.  
- For every group of pages, writes them into a new PDF.  
- Adds each split PDF into the `.zip` file.  

✅ Output: **ZIP file containing split PDFs**.

---

## 🔹 4. `remove_pages(file, pages_to_remove)`
**Purpose:**  
Remove specific pages from a PDF.  

**Steps in my code:**
- Parses the input (e.g., `"1,3,5-7"`) into a list of pages to delete.  
- Loops through all pages, skipping the ones listed.  
- Writes the new PDF without those pages.  

✅ Output: **cleaned PDF without unwanted pages**.

---

## 🔹 5. `compress_pdf(file)`
**Purpose:**  
Reduce the file size of a PDF.  

**Steps in my code:**
- Uses `page.compress_content_streams()` to shrink content streams.  
- Rewrites all pages into a new PDF.  
- Compares **original size** vs **compressed size**.  
- Calculates percentage of size reduction.  

✅ Output: **smaller PDF** + report of space saved.

---

## 🔹 6. `rotate_pages(file, rotation_angle, page_range="all")`
**Purpose:**  
Rotate pages in a PDF (90°, 180°, or 270°).  

**Steps in my code:**
- If `"all"` → rotate every page.  
- If custom (e.g., `"1,3-5"`) → rotate selected pages only.  
- Applies rotation with `page.rotate(rotation_angle)`.  
- Writes rotated pages into a new PDF.  

✅ Output: **rotated PDF**.

---

## 🔹 7. `extract_text(file)`
**Purpose:**  
Extract text content from a PDF.  

**Steps in my code:**
- Reads each page.  
- Uses `page.extract_text()` to pull out text.  
- Appends page numbers as separators (`--- Page X ---`).  

✅ Output: **plain text string** with all extracted text.

---

## 🔹 8. `add_watermark(file, watermark_text, opacity=0.3)`
**Purpose:**  
Add a diagonal **text watermark** (like "CONFIDENTIAL").  

**Steps in my code:**
- Creates a temporary watermark PDF in memory with `reportlab.canvas`.  
- Styles watermark (font, size, color, opacity, rotation).  
- Overlays watermark page on every page of the original PDF.  
- Saves new watermarked file.  

✅ Output: **PDF with watermark applied**.

---

## 🔹 9. `rearrange_pages(file, new_order)`
**Purpose:**  
Reorder pages of a PDF into a custom sequence.  

**Steps in my code:**
- Parses input order (e.g., `"3,1,4,2"`).  
- Validates that page numbers exist within the PDF.  
- Writes pages into the new sequence.  

✅ Output: **PDF with pages rearranged**.

---

# ✨ Summary
The `PDFToolkit` class turns Python + PyPDF2 + ReportLab into a full **PDF editor**:  
- Merge, split, compress, rotate  
- Extract text, add watermark, rearrange pages  
- Remove unwanted pages  

All results are written into **temporary files** for easy download in Colab.


In [ ]:
# Initialize the toolkit
toolkit = PDFToolkit()

# 🚀 Initializing the Toolkit

Before using any of the PDF functions, we need to **create an instance** of the `PDFToolkit` class.  

In [ ]:
# Create Gradio interface optimized for Colab
def create_interface():
    with gr.Blocks(title="PDF Toolkit - Personal iLovePDF Clone", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 📄 PDF Toolkit - Personal iLovePDF Clone")
        gr.Markdown("A comprehensive PDF manipulation tool running in Google Colab!")

        with gr.Tabs():
            # Merge PDFs Tab
            with gr.TabItem("🔗 Merge PDFs"):
                merge_files = gr.File(label="Upload PDF files to merge", file_count="multiple", file_types=[".pdf"])
                merge_btn = gr.Button("Merge PDFs", variant="primary")
                merge_output = gr.File(label="Download Merged PDF")
                merge_status = gr.Textbox(label="Status", interactive=False)

                merge_btn.click(
                    toolkit.merge_pdfs,
                    inputs=[merge_files],
                    outputs=[merge_output, merge_status]
                )

            # Split PDF Tab
            with gr.TabItem("✂️ Split PDF"):
                split_file = gr.File(label="Upload PDF to split", file_types=[".pdf"])
                split_pages = gr.Number(label="Pages per split", value=1, minimum=1)
                split_btn = gr.Button("Split PDF", variant="primary")
                split_output = gr.File(label="Download Split PDFs (ZIP)")
                split_status = gr.Textbox(label="Status", interactive=False)

                split_btn.click(
                    toolkit.split_pdf,
                    inputs=[split_file, split_pages],
                    outputs=[split_output, split_status]
                )

            # Remove Pages Tab
            with gr.TabItem("🗑️ Remove Pages"):
                remove_file = gr.File(label="Upload PDF", file_types=[".pdf"])
                pages_to_remove = gr.Textbox(
                    label="Pages to remove",
                    placeholder="e.g., 1,3,5-7 (page numbers separated by commas, ranges with hyphens)"
                )
                remove_btn = gr.Button("Remove Pages", variant="primary")
                remove_output = gr.File(label="Download Modified PDF")
                remove_status = gr.Textbox(label="Status", interactive=False)

                remove_btn.click(
                    toolkit.remove_pages,
                    inputs=[remove_file, pages_to_remove],
                    outputs=[remove_output, remove_status]
                )

            # Compress PDF Tab
            with gr.TabItem("🗜️ Compress PDF"):
                compress_file = gr.File(label="Upload PDF to compress", file_types=[".pdf"])
                compress_btn = gr.Button("Compress PDF", variant="primary")
                compress_output = gr.File(label="Download Compressed PDF")
                compress_status = gr.Textbox(label="Status", interactive=False)

                compress_btn.click(
                    toolkit.compress_pdf,
                    inputs=[compress_file],
                    outputs=[compress_output, compress_status]
                )

            # Rotate Pages Tab
            with gr.TabItem("🔄 Rotate Pages"):
                rotate_file = gr.File(label="Upload PDF", file_types=[".pdf"])
                rotation_angle = gr.Dropdown(
                    choices=[90, 180, 270],
                    label="Rotation angle",
                    value=90
                )
                rotate_pages = gr.Textbox(
                    label="Pages to rotate",
                    value="all",
                    placeholder="'all' or page numbers like '1,3,5-7'"
                )
                rotate_btn = gr.Button("Rotate Pages", variant="primary")
                rotate_output = gr.File(label="Download Rotated PDF")
                rotate_status = gr.Textbox(label="Status", interactive=False)

                rotate_btn.click(
                    toolkit.rotate_pages,
                    inputs=[rotate_file, rotation_angle, rotate_pages],
                    outputs=[rotate_output, rotate_status]
                )

            # Extract Text Tab
            with gr.TabItem("📝 Extract Text"):
                extract_file = gr.File(label="Upload PDF to extract text", file_types=[".pdf"])
                extract_btn = gr.Button("Extract Text", variant="primary")
                extracted_text = gr.Textbox(label="Extracted Text", lines=10, max_lines=20)

                extract_btn.click(
                    toolkit.extract_text,
                    inputs=[extract_file],
                    outputs=[extracted_text]
                )

            # Add Watermark Tab
            with gr.TabItem("💧 Add Watermark"):
                watermark_file = gr.File(label="Upload PDF", file_types=[".pdf"])
                watermark_text = gr.Textbox(label="Watermark text", placeholder="CONFIDENTIAL")
                watermark_opacity = gr.Slider(
                    label="Opacity",
                    minimum=0.1,
                    maximum=1.0,
                    value=0.3,
                    step=0.1
                )
                watermark_btn = gr.Button("Add Watermark", variant="primary")
                watermark_output = gr.File(label="Download Watermarked PDF")
                watermark_status = gr.Textbox(label="Status", interactive=False)

                watermark_btn.click(
                    toolkit.add_watermark,
                    inputs=[watermark_file, watermark_text, watermark_opacity],
                    outputs=[watermark_output, watermark_status]
                )

            # Rearrange Pages Tab
            with gr.TabItem("🔀 Rearrange Pages"):
                rearrange_file = gr.File(label="Upload PDF", file_types=[".pdf"])
                new_order = gr.Textbox(
                    label="New page order",
                    placeholder="e.g., 3,1,4,2 (page numbers in desired order)"
                )
                rearrange_btn = gr.Button("Rearrange Pages", variant="primary")
                rearrange_output = gr.File(label="Download Rearranged PDF")
                rearrange_status = gr.Textbox(label="Status", interactive=False)

                rearrange_btn.click(
                    toolkit.rearrange_pages,
                    inputs=[rearrange_file, new_order],
                    outputs=[rearrange_output, rearrange_status]
                )

    return demo

# 🎨 Creating the Gradio Interface (Optimized for Colab)

The function `create_interface()` builds a **user-friendly web app** inside Google Colab using **Gradio**.  
It connects the `PDFToolkit` methods to an interactive interface with tabs, buttons, and file uploaders.  
This essentially turns your Python code into a mini version of **iLovePDF**.

---

## 🔹 Function Overview

The structure of the function looks like this:

- `gr.Blocks` → container for the whole app.  
- `gr.Tabs` → organizes each PDF feature into its own tab.  
- Inside each `gr.TabItem`, we define:
  - **File uploaders** for input PDFs.  
  - **Buttons** to trigger actions.  
  - **Download outputs** for the processed PDFs.  
  - **Status messages** to show success or errors.  

At the end, `return demo` gives back the full Gradio app.

---

## 🔹 Components Used

- **`gr.Markdown`** → displays headers, titles, and descriptions.  
- **`gr.File`** → upload PDFs (input) or download results (output).  
- **`gr.Button`** → triggers PDF operations.  
- **`gr.Textbox`** → for user text input (page numbers, watermark text, status messages).  
- **`gr.Number`** → numeric input (e.g., pages per split).  
- **`gr.Slider`** → adjustable value (e.g., watermark opacity).  
- **`gr.Dropdown`** → choose options (e.g., rotation angles).  
- **`.click()`** → links a button to a backend function, with defined inputs and outputs.  

---

## 🔹 Tabs and Their Functions

### 1. 🔗 Merge PDFs
- Upload **two or more PDFs**.  
- Click **"Merge PDFs"** → calls `toolkit.merge_pdfs()`.  
- Outputs:  
  - Merged PDF (downloadable).  
  - Status message confirming success or error.  

---

### 2. ✂️ Split PDF
- Upload **a single PDF**.  
- Enter **pages per split** (e.g., `1` → split into single pages, `2` → split every 2 pages).  
- Click **"Split PDF"** → calls `toolkit.split_pdf()`.  
- Outputs:  
  - A `.zip` file containing all split parts.  
  - Status message with number of splits created.  

---

### 3. 🗑️ Remove Pages
- Upload **a single PDF**.  
- Enter pages to remove (e.g., `1,3,5-7`).  
- Click **"Remove Pages"** → calls `toolkit.remove_pages()`.  
- Outputs:  
  - Modified PDF with those pages removed.  
  - Status message confirming which pages were deleted.  

---

### 4. 🗜️ Compress PDF
- Upload **a single PDF**.  
- Click **"Compress PDF"** → calls `toolkit.compress_pdf()`.  
- Outputs:  
  - Compressed PDF file (downloadable).  
  - Status message with percentage of size reduced.  

---

### 5. 🔄 Rotate Pages
- Upload **a single PDF**.  
- Choose **rotation angle**: 90, 180, or 270 degrees.  
- Specify which pages to rotate (`all` or custom list like `1,3,5-7`).  
- Click **"Rotate Pages"** → calls `toolkit.rotate_pages()`.  
- Outputs:  
  - Rotated PDF.  
  - Status message with confirmation.  

---

### 6. 📝 Extract Text
- Upload **a single PDF**.  
- Click **"Extract Text"** → calls `toolkit.extract_text()`.  
- Outputs:  
  - Extracted text displayed in a textbox (per page).  

---

### 7. 💧 Add Watermark
- Upload **a single PDF**.  
- Enter watermark text (e.g., `CONFIDENTIAL`).  
- Adjust watermark **opacity** (0.1 = very faint, 1.0 = solid).  
- Click **"Add Watermark"** → calls `toolkit.add_watermark()`.  
- Outputs:  
  - Watermarked PDF (downloadable).  
  - Status message confirming the watermark was added.  

---

### 8. 🔀 Rearrange Pages
- Upload **a single PDF**.  
- Enter the new page order (e.g., `3,1,4,2`).  
- Click **"Rearrange Pages"** → calls `toolkit.rearrange_pages()`.  
- Outputs:  
  - Rearranged PDF file.  
  - Status message confirming the new order.  

---

## ✨ Why This Setup Works Great in Colab
- **All tools in one place** with easy navigation via tabs.  
- **Drag-and-drop file uploads** make it simple for non-programmers.  
- **Download links** allow saving processed PDFs locally.  
- **Separation of concerns** → interface handles inputs/outputs, while `PDFToolkit` does the PDF processing.  

---

## 🚀 How to Launch the App

Once the function is defined, create and launch the interface:

```python
demo = create_interface()
demo.launch()


In [ ]:
# Launch the interface
if __name__ == "__main__":
    demo = create_interface()
    # For Colab, use share=True to get a public link that works with Colab's environment
    demo.launch(
        share=True,  # Creates a public shareable link for Colab
        debug=True   # Helpful for debugging in Colab
    )